In [10]:
import os
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator


In [11]:
# Register train datasets
register_coco_instances("my_dataset_train", {}, "datasets/train/train.json", "datasets/train")

# Check dataset
metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

print("Classes:", metadata.thing_classes)  # should print ['objects', 'Nextar', 'Steam Cake']
print("Total images:", len(dataset_dicts))


WARNING [10/02 22:50:08 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/02 22:50:08 d2.data.datasets.coco]: Loaded 830 images in COCO format from datasets/train/train.json
Classes: ['objects', 'Nextar', 'Steam Cake']
Total images: 830


In [12]:
# Register val datasets
register_coco_instances("my_dataset_val", {}, "datasets/val/val.json", "datasets/val")

# Check dataset
metadata = MetadataCatalog.get("my_dataset_val")
dataset_dicts = DatasetCatalog.get("my_dataset_val")

print("Classes:", metadata.thing_classes)  # should print ['objects', 'Nextar', 'Steam Cake']
print("Total images:", len(dataset_dicts))


WARNING [10/02 22:50:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/02 22:50:09 d2.data.datasets.coco]: Loaded 75 images in COCO format from datasets/val/val.json
Classes: ['objects', 'Nextar', 'Steam Cake']
Total images: 75


In [13]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Dataset
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Pretrained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

# Training hyperparams
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.STEPS = (6000, 8000)
cfg.SOLVER.MAX_ITER = 10000

# ROI Heads
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3   # objects, Nextar, Steam Cake

# Input resizing (paper-backed defaults)
cfg.INPUT.MIN_SIZE_TRAIN = (640, 672, 704, 736, 768, 800)
cfg.INPUT.MAX_SIZE_TRAIN = 1333
cfg.INPUT.MIN_SIZE_TEST = 800
cfg.INPUT.MAX_SIZE_TEST = 1333

# Output directory
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [14]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[10/02 22:50:13 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[10/02 22:50:13 d2.engine.train_loop]: Starting training from iteration 0


E:\envs\dev\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4324.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/02 22:50:26 d2.utils.events]:  eta: 1:07:12  iter: 19  total_loss: 2.09  loss_cls: 1.333  loss_box_reg: 0.738  loss_rpn_cls: 0.001898  loss_rpn_loc: 0.008586    time: 0.3911  last_time: 0.3664  data_time: 0.1811  last_data_time: 0.0015   lr: 4.9953e-06  max_mem: 2224M
[10/02 22:50:34 d2.utils.events]:  eta: 1:07:44  iter: 39  total_loss: 2.239  loss_cls: 1.246  loss_box_reg: 0.9412  loss_rpn_cls: 0.006594  loss_rpn_loc: 0.01212    time: 0.3934  last_time: 0.3639  data_time: 0.0018  last_data_time: 0.0014   lr: 9.9902e-06  max_mem: 2224M
[10/02 22:50:42 d2.utils.events]:  eta: 1:07:45  iter: 59  total_loss: 2.003  loss_cls: 1.108  loss_box_reg: 0.8712  loss_rpn_cls: 0.002758  loss_rpn_loc: 0.007243    time: 0.3941  last_time: 0.3377  data_time: 0.0018  last_data_time: 0.0017   lr: 1.4985e-05  max_mem: 2224M
[10/02 22:50:50 d2.utils.events]:  eta: 1:07:37  iter: 79  total_loss: 1.742  loss_cls: 0.9337  loss_box_reg: 0.8003  loss_rpn_cls: 0.00299  loss_rpn_loc: 0.006689    time: 0.393

In [15]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("my_dataset_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
results = inference_on_dataset(trainer.model, val_loader, evaluator)
print(results)


WARNING [10/03 00:02:57 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [10/03 00:02:57 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/03 00:02:57 d2.data.datasets.coco]: Loaded 75 images in COCO format from datasets/val/val.json
[10/03 00:02:57 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/03 00:02:57 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[10/03 00:02:57 d2.data.common]: Serializing 75 elements to byte tensors and concatenating them all ...
[10/03 00:02:57 d2.data.common]: Serialized dataset takes 0.05 MiB
[10/03 00:02:57 d2.evaluation.evaluator]: Start inference on 75 batches
[10/03 00:03:02 d2.evaluation.evaluator]: Inference d